In [3]:
# ALWAYS RUN THIS CELL
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))


In [4]:
import os
from pathlib import Path

import paramiko
from scp import SCPClient

import getpass
import gc

/media/rich/OS/Users/Richard/Linux_stuff_on_OS/conda_envs/envs/suite2p/lib/python3.8/site-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [5]:
dir_github = '/media/rich/Home_Linux_partition/github_repos/'
import sys
sys.path.append(dir_github)

%load_ext autoreload
%autoreload 2
from s2p_on_o2 import util

In [6]:
# paramiko.util.log_to_file("paramiko.log")

----
# ==== Optional ====
### transfer raw data from local machine to `MICROSCOPE`

In [38]:
## Define directories for where the local data is and the remote directory where the contents of that folder should be copied to (MICROSCOPE)
dir_data_local = '/media/rich/bigSSD/tmp_data/'
dir_data_MICROSCOPE = '/n/files/Neurobio/MICROSCOPE/Rich/data/test'  ## contents of dir_data_local will be dumped into dir_data_MICROSCOPE

print(f'contents of    {dir_data_local}    will be copied to    {dir_data_MICROSCOPE}')

contents of    /media/rich/bigSSD/tmp_data/    will be copied to    /n/files/Neurobio/MICROSCOPE/Rich/data/test


In [6]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = util.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [ ]:
sftp = util.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=util.pw_decode(pw))

In [30]:
# sftp.mkdir_safe(dir_data_remote)
sftp.put_dir(dir_data_local, dir_data_MICROSCOPE)

uploading /media/rich/bigSSD/tmp_data/day0_00001_00001.tif   to   /n/files/Neurobio/MICROSCOPE/Rich/data/test/day0_00001_00001.tif
uploading /media/rich/bigSSD/tmp_data/day0_00001_00002.tif   to   /n/files/Neurobio/MICROSCOPE/Rich/data/test/day0_00001_00002.tif


------------
# ==== Start ====

## Prepare directories

In [7]:
## Define directories for where the local data is and the remote directory where the contents of that folder should be copied to
dir_data_MICROSCOPE = '/n/files/Neurobio/MICROSCOPE/Rich/data/test/'
dir_data_remote = '/n/data1/hms/neurobio/sabatini/rich/data/test'  ## contents of dir_data_local will be dumped into dir_data_remote


dir_S2pOutput_remote = '/n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/testRun'  ## outer directory. Final outputs will be in an inner directory
name_job = 'jobNum_'


path_dispatcher_local = Path(r'/media/rich/Home_Linux_partition/github_repos/s2p_on_o2/').resolve() / 'dispatcher.py'  ## path to the dispatcher.py file on local computer
path_dispatcher_remote = Path(r'/n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/') / 'dispatcher.py'  ## path to where you want the dispatcher.py file to be copied onto the server
path_s2pScript_remote = '/n/data1/hms/neurobio/sabatini/rich/github_repos/s2p_on_o2/remote_run_s2p.py'  ## path to a copy of the remote_run_s2p.py file within the s2p_on_o2 repo on the server

print(f'contents of    {dir_data_MICROSCOPE}    will be copied to    {dir_data_remote}')

contents of    /n/files/Neurobio/MICROSCOPE/Rich/data/test/    will be copied to    /n/data1/hms/neurobio/sabatini/rich/data/test


## Prepare username + password

In [8]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = util.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [9]:
sftp = util.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=util.pw_decode(pw))

In [10]:
## initialize ssh_transfer
ssh_t = util.ssh_interface(
    nbytes_toReceive=4096,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=util.pw_decode(pw),
    passcode_method=1,
    verbose=1,
)

Duo two-factor login for rh183

Enter a passcode or select one of the following options:

 1. Duo Push to XXX-XXX-4151
 2. Phone call to XXX-XXX-4151
 3. SMS passcodes to XXX-XXX-4151

Passcode or option (1-3): 
1



Pushed a login request to your device...

Success. Logging you in...

Welcome to O2 (Orchestra 2)!

   ___    ____
  / _ \  |___ \
 | | | |   __) |
 | |_| |  / __/
  \___/  |_____|


You've landed on transfer03 which is a
4 core system with 7.62 GiB memory
running kernel 3.10.0 born on 2020-07-31

==== O2 ===================================================================

  News (Jan 3 2022)
  

  +-------------------------------------------------------------------------------+
  | ##################### RC Core launched in July 2021 ######################### |
  |                                                                               |
  | Since July 2021, researchers whose PI does not have a primary or secondary    |
  | faculty appointment in an HMS Basic or Soci

In [11]:
## initialize ssh_compute
ssh_c = util.ssh_interface(
    nbytes_toReceive=4096,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=util.pw_decode(pw),
    passcode_method=1,
    verbose=1,
)

Duo two-factor login for rh183

Enter a passcode or select one of the following options:

 1. Duo Push to XXX-XXX-4151
 2. Phone call to XXX-XXX-4151
 3. SMS passcodes to XXX-XXX-4151

Passcode or option (1-3): 
1



Pushed a login request to your device...

Success. Logging you in...
Welcome to O2 (Orchestra 2)!

   ___    ____
  / _ \  |___ \
 | | | |   __) |
 | |_| |  / __/
  \___/  |_____|


You've landed on login01 which is a
8 core system with 15.50 GiB memory
running kernel 3.10.0 born on 2020-07-07

==== O2 ===================================================================

  News (Jan 3 2022)
  

  +-------------------------------------------------------------------------------+
  | ##################### RC Core launched in July 2021 ######################### |
  |                                                                               |
  | Since July 2021, researchers whose PI does not have a primary or secondary    |
  | faculty appointment in an HMS Basic or Social 

## Prepare commands

In [12]:
prompt_snip = f'[{username}'

commands = {
    'make_dir': f"mkdir {dir_data_remote}",
    'copy_data': f"cp -r {str(Path(dir_data_MICROSCOPE).resolve())}/. {dir_data_remote}",
    'dispatch_s2p': f"python {str(path_dispatcher_remote)} {dir_S2pOutput_remote} {path_s2pScript_remote} {name_job} {dir_data_remote}"  ## dispatcher expecting these args as inputs
}
display(commands)

{'make_dir': 'mkdir /n/data1/hms/neurobio/sabatini/rich/data/test',
 'copy_data': 'cp -r /n/files/Neurobio/MICROSCOPE/Rich/data/test/. /n/data1/hms/neurobio/sabatini/rich/data/test',
 'dispatch_s2p': 'python /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/dispatcher.py /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/testRun /n/data1/hms/neurobio/sabatini/rich/github_repos/s2p_on_o2/remote_run_s2p.py jobNum_ /n/data1/hms/neurobio/sabatini/rich/data/test'}

## Upload `dispatcher.py` file

In [13]:
sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));

## Copy data from `MICROSCOPE` to `data1`

In [41]:
ssh_t.send_receive(commands['make_dir']);

t
(base) [rh183@transfer03 ~]$ 


In [42]:
ssh_t.send(commands['copy_data'])
ssh_t.expect(str_success=prompt_snip, total_timeout=120, verbose=True);

cp -r /n/files/Neurobio/MICROSCOPE/Rich/data/tes
 /n/data1/hms/neurobio/sabatini/rich/data/test

(base) [rh183@transfer03 ~]$ 


## Dispatch `remote_run_s2p.py`

In [14]:
ssh_c.send(commands['dispatch_s2p']);

ssh_c.expect(str_success=prompt_snip);

ite2p_output/dispatcher.py /n/data1/hmsich/analysis/su 
output/testRun /n/data1/hms/neurobio/sabatini/rich/gith
_run_s2p.py jobNu2/remote 
m_ /n/data1/hms/neurobio/sabatini/rich/data/test

/n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/dispatcher.py /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/testRun /n/data1/hms/neurobio/sabatini/rich/github_repos/s2p_on_o2/remote_run_s2p.py jobNum_ /n/data1/hms/neurobio/sabatini/rich/data/test

Submitting job: jobNum_ 0

Submitted batch job 53419173

(base) [rh183@login01 ~]$ 


### check on job

In [15]:
ssh_c.send(cmd=f'squeue -u {username} --Format=jobid,name,partition,state,timeused,starttime,timelimit,reasonlist,minmemory,mincpus,gres');
ssh_c.expect(str_success=f"[{username}");

squeue -u rh183 --Format=jobid,name,partition,s
meused,starttime,timelimit,reasonlist,minmemory,mincpus,gres

JOBID               NAME                PARTITION           STATE               TIME                START_TIME          TIME_LIMIT          NODELIST(REASON)    MIN_MEMORY          MIN_CPUS            TRES_PER_NODE       

53419173            suite2p             priority            PENDING             0:00                N/A                 10:00               (None)              32G                 20                  N/A                 
(base) [rh183@login01 ~]$ 


In [17]:
ssh_c.send(cmd=f"cat {str(Path(dir_S2pOutput_remote).resolve() / (name_job+'0') / 'print*')}")
ssh_c.expect(str_success=f"[{username}");

cat /n/data1/hms/neurobi
2p_output/testRun/jobNum_0/print*

cat: /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/testRun/jobNum_0/print*: No such file or directory
(base) [rh183@login01 ~]$ 


In [27]:
flag_COMPLETE=False
while flag_COMPLETE==False:
    ssh_c.send(cmd=f"cat {str(Path(dir_S2pOutput_remote).resolve() / (name_job+'0') / 'log.txt')}")
    # ssh_c.expect(str_success=f"[{username}")
    flag_COMPLETE = ssh_c.expect(str_success=f"RUN COMPLETE", total_timeout=1, verbose=False)[1]
    time.sleep(1)

print(f'RUN COMPLETE!!!     {time.ctime()}')

RUN COMPLETE!!!     Tue May 17 14:52:20 2022


## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()

----
# ==== Optional ====
### transfer s2p outputs back from `data1` to `MICROSCOPE`

## Prepare username + password

In [7]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = util.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `ssh` object

In [48]:
## initialize ssh_transfer
ssh_t = util.ssh_interface(
    nbytes_toReceive=4096,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=util.pw_decode(pw),
    passcode_method=1,
    verbose=1,
)

Duo two-factor login for rh183

Enter a passcode or select one of the following options:

 1. Duo Push to XXX-XXX-4151
 2. Phone call to XXX-XXX-4151
 3. SMS passcodes to XXX-XXX-4151

Passcode or option (1-3): 
1



Pushed a login request to your device...

Success. Logging you in...
Welcome to O2 (Orchestra 2)!

   ___    ____
  / _ \  |___ \
 | | | |   __) |
 | |_| |  / __/
  \___/  |_____|


You've landed on transfer08 which is a
4 core system with 7.62 GiB memory
running kernel 3.10.0 born on 2021-06-23

==== O2 ===================================================================

  News (Jan 3 2022)
  

  +-------------------------------------------------------------------------------+
  | ##################### RC Core launched in July 2021 ######################### |
  |                                                                               |
  | Since July 2021, researchers whose PI does not have a primary or secondary    |
  | faculty appointment in an HMS Basic or Socia

In [33]:
dir_S2pOutput_remote = dir_S2pOutput_remote  ## from above
dir_s2pOutput_MICROSCOPE = str(Path(dir_data_MICROSCOPE).resolve() / 'suite2p_o2_output')  ## from above
print(f'contents of    {dir_data_remote}    will be copied to    {dir_s2pOutput_MICROSCOPE}')

contents of    /n/data1/hms/neurobio/sabatini/rich/data/test    will be copied to    /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output


In [34]:
commands = {
    'make_dir': f"mkdir {dir_s2pOutput_MICROSCOPE}",
    'copy_s2p': f"cp -r {str(Path(dir_S2pOutput_remote).resolve())}/. {dir_s2pOutput_MICROSCOPE}",
}
display(commands)

{'make_dir': 'mkdir /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output',
 'copy_s2p': 'cp -r /n/data1/hms/neurobio/sabatini/rich/analysis/suite2p_output/testRun/. /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output'}

In [35]:
ssh_t.send_receive(commands['make_dir']);

uite2p_o2_output
(base) [rh183@transfer03 ~]$ 


In [36]:
ssh_t.send(commands['copy_s2p'])
ssh_t.expect(str_success=prompt_snip, total_timeout=120, verbose=True);

ouite2p_output/testRun/. /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_ 
tput

(base) [rh183@transfer03 ~]$ 


## Clean up

In [ ]:
## delete password
del pw
gc.collect()
    
## close ssh
ssh_t.close()
del ssh_t
gc.collect()

----
# ==== Optional ====
### transfer s2p outputs from `MICROSCOPE` to local machine

In [39]:
dir_s2pOutput_MICROSCOPE = dir_s2pOutput_MICROSCOPE  ## from above
dir_s2pOutput_local = str(Path(dir_data_local).resolve() / 'suite2p_o2_output')
print(f'contents of    {dir_s2pOutput_MICROSCOPE}    will be copied to    {dir_s2pOutput_local}')

contents of    /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output    will be copied to    /media/rich/bigSSD/tmp_data/suite2p_o2_output


In [57]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = util.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [40]:
sftp = util.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=util.pw_decode(pw))

In [41]:
sftp.get_dir(dir_s2pOutput_MICROSCOPE, dir_s2pOutput_local)

downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/params.json   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/params.json
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/print_log_53419173.log   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/print_log_53419173.log
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/batch_run_output.png   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/batch_run_output.png
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/log.txt   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/log.txt
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/sbatch_config.sh   to   /media/rich/bigSSD/tmp_data/suite2p_o2_output/jobNum_0/sbatch_config.sh
downloading /n/files/Neurobio/MICROSCOPE/Rich/data/test/suite2p_o2_output/jobNum_0/remote_run_s2p.py   to   /medi

## Clean up

In [43]:
## delete password
del pw
gc.collect()

## close sftp
sftp.close()
del sftp

In [42]:
ssh_t.close()
ssh_c.close()